<a href="https://colab.research.google.com/github/jellyontherock/AI/blob/master/20220504_%EB%8D%B0%EC%9D%B4%EC%BD%98%EC%86%8C%EB%B9%84%EC%9E%90%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import lightgbm as lgbm
import os

from sklearn.preprocessing import LabelEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# train, test 불러오기

train = pd.read_csv('/content/drive/MyDrive/머신러닝/데이콘/소비자 데이터 기반 소비 예측 경진대회/train (1).csv')
train = pd.DataFrame(train)

test = pd.read_csv('/content/drive/MyDrive/머신러닝/데이콘/소비자 데이터 기반 소비 예측 경진대회/test.csv')
test = pd.DataFrame(test)

In [4]:
Education_train = np.array(train['Education'])
Marital_Status_train = np.array(train['Marital_Status'])

Education_test = np.array(test['Education'])
Marital_Status_test = np.array(test['Marital_Status'])

In [5]:
# 비정형 Education, Marital_Status 숫자로 치환해주기

# 라벨 인코더 생성
encoder = LabelEncoder()

# X_train 데이터를 이용 피팅하고 라벨숫자로 변환한다
Education_fit = encoder.fit(Education_train)
Education_encoder_train = encoder.transform(Education_train)
print(encoder.classes_)

['2n Cycle' 'Basic' 'Graduation' 'Master' 'PhD']


In [6]:
encoder.fit(Marital_Status_train)
Marital_Status_encoder_train = encoder.transform(Marital_Status_train)
print(encoder.classes_)

['Absurd' 'Alone' 'Divorced' 'Married' 'Single' 'Together' 'Widow' 'YOLO']


In [7]:
Education_fit_test = encoder.fit(Education_test)
Education_encoder_test = encoder.transform(Education_test)
print(encoder.classes_)

['2n Cycle' 'Basic' 'Graduation' 'Master' 'PhD']


In [8]:
Marital_Status_fit_test = encoder.fit(Marital_Status_test)
Marital_Status_encoder_test = encoder.transform(Marital_Status_test)
print(encoder.classes_)

['Absurd' 'Alone' 'Divorced' 'Married' 'Single' 'Together' 'Widow' 'YOLO']


In [9]:
train['Education'] = Education_encoder_train
train['Marital_Status'] = Marital_Status_encoder_train

test['Education'] = Education_encoder_test
test['Marital_Status'] = Marital_Status_encoder_test

In [13]:
train['Dt_Customer'] = pd.to_datetime(train['Dt_Customer'])
train['yyyy'] = train['Dt_Customer'].dt.strftime('%Y')
train['mm'] = train['Dt_Customer'].dt.strftime('%m')
train['dd'] = train['Dt_Customer'].dt.strftime('%d')

In [16]:
train_y = train[['target']].astype(float)
train_x = train[['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'yyyy', 'mm', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response']].astype(float)

In [24]:
test['Dt_Customer'] = pd.to_datetime(test['Dt_Customer'])
test['yyyy'] = test['Dt_Customer'].dt.strftime('%Y')
test['mm'] = test['Dt_Customer'].dt.strftime('%m')

In [25]:
test_x = test[['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'yyyy', 'mm', 'Recency', 'NumDealsPurchases',
       'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
       'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5',
       'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response']].astype(float)

In [34]:
model_LGBM = lgbm.LGBMClassifier()
model_LGBM.fit(train_x,train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier()

In [35]:
LGBM_pred = model_LGBM.predict(test_x)

In [36]:
LGBM_pred 

array([860.,  64., 907., ...,  71., 235., 120.])

In [65]:
submission = pd.read_csv('/content/drive/MyDrive/머신러닝/데이콘/소비자 데이터 기반 소비 예측 경진대회/sample_submission.csv')

In [67]:
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [68]:
submission['target'] = LGBM_pred
submission['id'] = test['id'] 

In [69]:
submission = pd.DataFrame(submission)

In [70]:
submission['target'] = submission['target'].astype(int)

In [71]:
submission

,id,target
0,0,860
1,1,64
2,2,907
3,3,1371
4,4,1497
...,...,...
1103,1103,62
1104,1104,35
1105,1105,71
1106,1106,235


In [77]:
submission.to_csv('/content/drive/MyDrive/머신러닝/데이콘/소비자 데이터 기반 소비 예측 경진대회/submission_jelly.csv',index=False)
